In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


# Overview
this is for making bentmark of model. I learned a lot from below notebook.

test a lot.

[ICR | Simple EDA & Baseline](https://www.kaggle.com/code/datafan07/icr-simple-eda-baseline)

In [2]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [3]:
train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

In [4]:
num_cols = train.columns.tolist()[1:-1]
cat_cols = 'EJ'
num_cols.remove(cat_cols)

In [5]:
from sklearn.preprocessing import LabelEncoder
# Create a LabelEncoder object.
encoder = LabelEncoder()
# Transform the data.
train[cat_cols] = encoder.fit_transform(train[cat_cols])
test[cat_cols] = encoder.transform(test[cat_cols])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
oof = np.zeros((len(train), 2))

!pip install iterative-stratification
from tqdm.auto import tqdm
import catboost as cb

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

final_preds = []

params={
    'iterations':10000,
    'learning_rate':0.005,
    'early_stopping_rounds':1000,
    'auto_class_weights':'Balanced',
    'loss_function':'MultiClass',
    'eval_metric':'MultiClass:use_weights=True',
    'random_seed':42,
    'use_best_model':True,
    'l2_leaf_reg':1,
    'max_ctr_complexity':15,
    'max_depth':10,
    "grow_policy":'Lossguide',
    'max_leaves':64,
    "min_data_in_leaf":40,
    }

In [7]:
for train_index,val_index in skf.split(train, greeks.iloc[:,1:-1]):

    X_train, X_val = train.loc[train_index, num_cols + [cat_cols]], train.loc[val_index, num_cols + [cat_cols]]
    y_train, y_val = train.loc[train_index, 'Class'], train.loc[val_index, 'Class']
    
    
    model = cb.CatBoostClassifier(**params)
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)], verbose=1000)
    preds = model.predict_proba(X_val)
    oof[val_index, :] = preds
    final_preds.append(model.predict_proba(test.iloc[:,1:]))

0:	learn: 0.6907267	test: 0.6912612	best: 0.6912612 (0)	total: 69.4ms	remaining: 11m 34s
1000:	learn: 0.0998772	test: 0.3644460	best: 0.3633853 (992)	total: 10.9s	remaining: 1m 37s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.3633853116
bestIteration = 992

Shrink model to first 993 iterations.
0:	learn: 0.6909054	test: 0.6911446	best: 0.6911446 (0)	total: 11.6ms	remaining: 1m 55s
1000:	learn: 0.1053599	test: 0.2726799	best: 0.2726799 (1000)	total: 10.9s	remaining: 1m 38s
2000:	learn: 0.0230848	test: 0.2533569	best: 0.2511949 (1708)	total: 21.9s	remaining: 1m 27s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.2511949429
bestIteration = 1708

Shrink model to first 1709 iterations.
0:	learn: 0.6907639	test: 0.6910948	best: 0.6910948 (0)	total: 11.2ms	remaining: 1m 52s
1000:	learn: 0.1025662	test: 0.2475667	best: 0.2475667 (1000)	total: 11s	remaining: 1m 38s
2000:	learn: 0.0239101	test: 0.1919123	best: 0.1919123 (2000)	total: 21.7s	remaining

In [8]:
from sklearn.metrics import log_loss

def balance_loglossv2(y_true, y_pred):
    target_mean = y_true.mean()
    w0 = 1/(1-target_mean)
    w1 = 1/target_mean
    sample_weight = [w0 if y == 0 else w1 for y in y_true]
    loss = log_loss(y_true, y_pred, sample_weight=sample_weight)
    
    return loss

print(balance_loglossv2(train['Class'], oof))

0.2667928240388374


In [9]:
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample_submission[['class_0','class_1']] = np.mean(final_preds, axis=0)
sample_submission.to_csv('submission.csv',index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.774959,0.225041
1,010ebe33f668,0.774959,0.225041
2,02fa521e1838,0.774959,0.225041
3,040e15f562a2,0.774959,0.225041
4,046e85c7cc7f,0.774959,0.225041
